# Week 5 Assignment: Songs Recommender System

Richard Yang

In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in the data

df = pd.read_csv('radio_songs.csv')
df.head()

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1. Collaborative Filtering



### A. Use this user-item matrix to: Recommend 10 songs to users who have listened to 'u2' and 'pink floyd'. Use item-item collaborative filtering to find songs that are similar using spatial distance with cosine. Since this measures the distance you need to subtract from 1 to get similarity as shown below.

In [178]:
# from scipy.spatial.distance import cosine
import math

# Define a function to calculate the cosine distance between two vectors

def cosine_defined(x, y):
    # Calculate the dot product and the norm of the vectors
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cosinedist = 1- dot_product / ((norm_x * norm_y) ) # prevent division by zero
    return cosinedist

# Define a function to calculate the cosine similarity between two vectors
def cosine_similarity(a, b):
    return 1 - cosine_defined(a, b)  




If the user has listened to 'u2' and 'pink floyd', then the top 10 songs that are most similar to the ones they have listened to are:

Lets' first do some data exploration. select U2 and Pink Floyd's columns from the original dataframe, store them in a new dataframe

In [179]:
df_u2_pink_floyd = df[['u2', 'pink floyd']]
df_u2_pink_floyd['combine'] = df_u2_pink_floyd['u2'] + df_u2_pink_floyd['pink floyd']
df_u2_pink_floyd = df_u2_pink_floyd.sort_values(by='combine', ascending=False)
df_u2_pink_floyd

,u2,pink floyd,combine
53,0,1,1
65,0,1,1
54,0,1,1
77,1,0,1
21,1,0,1
...,...,...,...
32,0,0,0
30,0,0,0
29,0,0,0
28,0,0,0


There is no '2' Label in 'Combine' Column. No one listened to both 'u2' and 'pink floyd' 

For all users who have listend to 'u2':

In [180]:
# # calculate the cosine similarity between song u2 all other songs

df_songs = df.iloc[:, 1:]
song_names = df_songs.columns
u2_list = {}
for columns in song_names:
    sum_similarity = cosine_similarity(df_songs['u2'], df_songs[columns])
    if not math.isnan(sum_similarity):
        u2_list[columns] = sum_similarity

# Create a dataframe with the cosine similarity values，sort the values in descending order

df_u2 = pd.DataFrame.from_dict(u2_list, orient='index', columns=['cosine_similarity'])
df_u2 = df_u2.sort_values(by='cosine_similarity', ascending=False)
df_u2.value_counts().head(8)

df_u2.iloc[1:11, :]

,cosine_similarity
misfits,0.500000
robbie williams,0.500000
green day,0.433013
depeche mode,0.408248
peter fox,0.377964
dire straits,0.353553
kelly clarkson,0.353553
madonna,0.353553
johnny cash,0.353553
enter shikari,0.353553


For all users who have listend to 'pink floyd':

In [181]:
# # calculate the cosine similarity between song pl all other songs
pl_list = {}
for columns in song_names:
    sum_similarity = cosine_similarity(df_songs['pink floyd'], df_songs[columns])
    if not math.isnan(sum_similarity):
        pl_list[columns] = sum_similarity

# Create a dataframe with the cosine similarity values，sort the values in descending order

df_pl = pd.DataFrame.from_dict(pl_list, orient='index', columns=['cosine_similarity'])
df_pl = df_pl.sort_values(by='cosine_similarity', ascending=False)
df_pl.value_counts().head(8)

df_pl.iloc[1:11, :]



,cosine_similarity
genesis,0.577350
sonic syndicate,0.408248
led zeppelin,0.408248
queen,0.408248
david bowie,0.408248
funeral for a friend,0.408248
hans zimmer,0.408248
coldplay,0.348155
maria mena,0.333333
howard shore,0.333333


Calculate a total similarity score for 'u2' and 'pink floyd' 

In [182]:
# for loop to calculate the cosine similarity between song u2 all other songs, and cosine similarity between song pink floyd all other songs, excluding nan values
cos_sim_list = {}
u2 = df[['u2']]
pink_floyd = df[['pink floyd']]

for columns in song_names:
    sum_similarity = cosine_similarity(df_songs['u2'], df_songs[columns])\
        +cosine_similarity(df_songs['pink floyd'], df_songs[columns])
    if not math.isnan(sum_similarity):
        cos_sim_list[columns] = sum_similarity

# Create a dataframe with the cosine similarity values，sort the values in descending order

df_cos_sim = pd.DataFrame.from_dict(cos_sim_list, orient='index', columns=['cosine_similarity'])
df_cos_sim = df_cos_sim.sort_values(by='cosine_similarity', ascending=False)
df_cos_sim.iloc[2:12]

,cosine_similarity
robbie williams,0.788675
genesis,0.577350
johnny cash,0.557678
misfits,0.500000
audioslave,0.481806
foo fighters,0.475589
pearl jam,0.439826
green day,0.433013
depeche mode,0.408248
hans zimmer,0.408248


### B. Find user most similar to user 1606. Use user-user collaborative filtering with cosine similarity. List the recommended songs for user 1606 (Hint: find the songs listened to by the most similar user).

In [183]:
# transpose the dataframe df and delete the first column

df_transpose = df.T[1:]
df_transpose.columns = df.T.iloc[0]

# calculate the cosine similarity between user 1606 and all other users

user_1606 = {}
for columns in df_transpose.columns:
    sum_similarity = cosine_similarity(df_transpose[1606], df_transpose[columns])
    if not math.isnan(sum_similarity):
        user_1606[columns] = sum_similarity

# Create a dataframe with the cosine similarity values，sort the values in descending order

df_user_1606 = pd.DataFrame.from_dict(user_1606, orient='index', columns=['cosine_similarity'])
df_user_1606 = df_user_1606.sort_values(by='cosine_similarity', ascending=False)
df_user_1606.value_counts().head(8)

df_user_1606.iloc[1:10]


,cosine_similarity
1144,0.277350
144,0.223607
1334,0.150756
1509,0.144338
890,0.129099
1259,0.125000
648,0.121268
1174,0.106600
504,0.106600


In [184]:
user1144 = df[df.user == 1144].T
print('Recommended songs for user 1606:\n') 
list(user1144[user1144.iloc[:,0] == 1].index)

Recommended songs for user 1606:



['beastie boys',
 'bob dylan',
 'bob marley & the wailers',
 'david bowie',
 'elvis presley',
 'eric clapton',
 'johnny cash',
 'pearl jam',
 'pink floyd',
 'the beatles',
 'the doors',
 'the rolling stones',
 'tom waits']

### C. How many of the recommended songs has already been listened to by user 1606?

In [185]:
user1606 = df[df.user == 1606].T
user1606df = user1606[user1606.iloc[:,0] == 1]
user1144df = user1144[user1144.iloc[:,0] == 1]

# find the common songs between user 1606 and user 1144

common_songs = user1606df.index.intersection(user1144df.index)
common_songs

Index(['elvis presley', 'the beatles'], dtype='object')

### D. Use a combination of user-item approach to build a recommendation score for each song for each user using the following steps for each user-

- For each song for the user row, get the top 10 similar songs and their similarity score.
- For each of the top 10 similar songs, get a list of the user purchases
- Calculate a recommendation score as follows: 
$$Recommendation Score = \sum\frac{(Purchase History * Similarity Score)}{Similarity Score} $$
- What are the top 5 song recommendations for user 1606?

In [214]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
import math

def find_similar_songs(song_name, song_df):
    song_df = song_df.iloc[:, 1:]
    column_names = song_df.columns
    song_col = np.array(song_df[song_name])
    similarity_df = pd.DataFrame(columns=['songs', 'similarity_score'])
    for col_name in column_names:
        songs = np.array(song_df[col_name])
        similarity = cosine_similarity(song_col, song_df[col_name])
        if pd.notnull(similarity):
            similarity_df = similarity_df.append({'songs': col_name, 'similarity_score': similarity}, ignore_index=True)
    output_df = similarity_df.sort_values(by='similarity_score', ascending=False).iloc[1:11,:]
    return output_df # return top 10 similar songs

# test the function
# call the function to get the top 10 similar songs for u2

output_df = find_similar_songs('u2', df)
output_df 


,songs,similarity_score
167,misfits,0.5
209,robbie williams,0.5
107,green day,0.433013
69,depeche mode,0.408248
190,peter fox,0.377964
74,dire straits,0.353553
141,kelly clarkson,0.353553
155,madonna,0.353553
130,johnny cash,0.353553
87,enter shikari,0.353553


In [215]:
# return a dictionary of item name and recommend score
def recommend_score(user_id, dataset):

    all_items = dataset.iloc[:,1:].columns # remove user column
    output_item_scores = {}

    for item in all_items:
        # calculate the recommendation score for the songs
        df_tmp = find_similar_songs(item, dataset)
        top10_items = list(zip(df_tmp['items'], df_tmp['similarity_score']))
        recommend_system_score = 0
        if len(top10_items) > 0:
            top10_name = [x[0] for x in top10_items]
            top10_score = [x[1] for x in top10_items]
            score_sum = sum(top10_score)
            user_df_tmp = dataset[dataset['user'] == user_id]
            for idx, item in enumerate(top10_name):
                recommend_system_score += user_df_tmp[item].values[0] * top10_score[idx]
            recommend_system_score = recommend_system_score / score_sum
        
        output_item_scores[item] = recommend_system_score

    return output_item_scores

In [216]:
recommend1606 = recommend_score(1606, df)

Top 5 song recommendations for user 1606:

In [241]:
recommend1606_result = pd.DataFrame.from_dict(recommend1606, orient='index', columns=['recommend_system_score'])
recommend1606_result = recommend1606_result.sort_values(by='recommend_system_score', ascending=False)
recommend1606_result = recommend1606_result.round(4)
recommend1606_result.head(5)

,recommend_system_score
elvis presley,0.2893
abba,0.2390
eric clapton,0.2027
frank sinatra,0.2011
howard shore,0.1717


## 2. Conceptual Questions

 #### 2.1 Name 2 other similarity measures that you can use instead of cosine similarity above.
<p> Euclidean Distance: It is calculated as the square root of the sum of the squared differences between the coordinates of two points. Euclidean distance is often used in clustering algorithms and image processing. 

$$\sqrt{\sum_{i=1}^n (x_i-y_i)^2} 



The Manhattan distance (sometimes also called Taxicab distance) metric is related to the Euclidean distance, but instead of calculating the shortest diagonal path ("beeline") between two points, it calculates the distance based on gridlines. The Manhattan distance was named after the block-like layout of the streets in Manhattan.

$$\left(\sum_{i=1}^n |x_i-y_i|^p\right)^{1/p}

Pearson Correlation: This measures the linear correlation between two variables, which can be positive, negative, or zero. It is often used to determine the strength and direction of a relationship between two variables. Pearson correlation ranges from -1 to +1, where -1 indicates a perfectly negative correlation, 0 indicates no correlation, and +1 indicates a perfectly positive correlation. Pearson correlation is often used in recommender systems to determine the similarity between users or items.

$$\rho = \frac{\text{cov}(X,Y)}{\sigma_x \sigma_y}

#### 2.2 What is needed to build a Content-Based Recommender system?

We need the following to build a Content-Based Recommender system:

- Item Features: A set of items with features that can be used to measure their similarity.

- User Profile: A representation of the user's preferences or interests in terms of the item features. 

- Similarity Measure: A way to measure the similarity between items based on their features.This could be a distance metric (e.g., Euclidean distance), a similarity coefficient (e.g., cosine similarity), or a kernel function (e.g., radial basis function).

- Recommendation Algorithm: A way to use the item features, user profile, and similarity measure to generate a list of recommended items for the user. This could be as simple as selecting the items that are most similar to the user's profile, or it could involve more sophisticated machine learning models that learn to predict the user's preferences based on their historical data.

- Evaluation Metrics: A way to measure the effectiveness of the recommendation algorithm in terms of how well it predicts the user's preferences. This could involve splitting the data into training and test sets, and using metrics such as accuracy, precision, recall, or F1-score to evaluate the performance of the system.

#### 2.3 Name 2 methods to evaluate your recommender system.

- Split the data into training and test sets, and use metrics such as accuracy, precision, recall, or F1-score to evaluate the performance of the system.

- Use cross-validation to evaluate the performance of the system. Cross-validation is a technique for evaluating the performance of a model on a dataset by splitting the dataset into k subsets, or folds. The model is trained on k-1 folds, and then evaluated on the remaining fold. This process is repeated k times, with each fold used exactly once as the test set. The performance of the model is then averaged over the k folds. This is useful for evaluating the performance of a model on new data, and it can also be used to compare the performance of different models.
